In [25]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from pandas import get_dummies, DataFrame

In [2]:


f=open('../Case Data/case1Data.txt','r')
data=f.read()
data1=data.split('\n')
arr=[]
for i in range(101):
    arr.append(data1[i].split(', '))
    


In [3]:
header=arr[0]
arr=arr[1:]
number_cols=[x[0:96] for x in arr]
class_cols=[x[96:] for x in arr]
y=np.array([x[0] for x in number_cols],dtype='float64')
x=np.array([x[1:] for x in number_cols],dtype='float64')
x_class=np.array(class_cols)

# one hot encoding





In [34]:
# # nan stuff

# c1=x_class[:,0]
# c2=x_class[:,1]
# c3=x_class[:,2]
# c4=x_class[:,3]
# c5=x_class[:,4]

# # for i in [c1,c2,c3,c4,c5]: 
# #     unique, counts = np.unique(i, return_counts=True)
# #     print (np.asarray((unique, counts)).T)

# count=0
# for i in range(101):
#     print(x_class[i])
# print(count)

In [4]:
# KNN
x_class_no_nans = x_class.copy()
def get_x_and_y(i):
    nans=np.where(i=='NaN')
    x_tmp = np.delete(x.copy(),nans,axis=0)
    y_tmp = np.delete(i.copy(),nans)
    return (x_tmp,y_tmp)

ns = range(1,25) #[1,3,5,10,20,60]
times_to_repeat = 200 
accuracy = np.zeros((x_class.shape[1],len(ns)))

for n_index, n in enumerate(ns):
    for col_index, col in enumerate(x_class.T):
        err = 0
        for loop in range(times_to_repeat):
            x_tmp,y_tmp = get_x_and_y(col)
            x_tmp, y_tmp = resample(x_tmp, y_tmp,n_samples=100)
            x_train, x_test,y_train, y_test = train_test_split(x_tmp,y_tmp,test_size=0.25)
            knn = KNeighborsClassifier(n_neighbors=n).fit(x_train,y_train)
            err += knn.score(x_test,y_test)
        err /= times_to_repeat
        accuracy[col_index, n_index] = err

In [5]:
# y_pred = knn.predict(x[nans])

for col_index, col in enumerate(x_class.T):
    best_k = np.argmax(accuracy[col_index]) + 1
    x_tmp,y_tmp = get_x_and_y(col)
    nans=np.where(col=='NaN')
    y_pred = KNeighborsClassifier(n_neighbors=best_k).fit(x_tmp,y_tmp).predict(x[nans])
    x_class_no_nans[nans,col_index] = y_pred

In [6]:
x_classes_one_hot = get_dummies(DataFrame(x_class_no_nans)).to_numpy()

x_knn = np.concatenate((x,x_classes_one_hot),axis=1)

x_one_hot = np.concatenate((x,get_dummies(DataFrame(x_class)).to_numpy()), axis=1)

In [7]:

def rmse(y_true,y_pred):
    return np.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
   

def train_model(model,x,y=y):
    #Linear Regression
    x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=0)
    m = model.fit(x_train, y_train)
    y_pred = m.predict(x_test)
    return rmse(y_test,y_pred)


lin_reg_rmse = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x_knn)
lin_reg_rmse_no_classes = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x)
lin_reg_rmse_no_knn = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x_one_hot)

In [15]:
times_to_repeat = 200 

def calc_accuracy(model_func, alpha, x_one=x_one_hot):
    accuracy = np.zeros((len(alpha)))
    for loop in range(times_to_repeat):
        err = 0
        for n_index, n in enumerate(alpha):
            x_train, x_test,y_train, y_test = train_test_split(x_one,y,test_size=0.25)
            model = model_func(n).fit(x_train,y_train)
            y_pred =  model.predict(x_test)
            accuracy[n_index] += (np.sum(y_test - y_pred)**2) / len(y_test)
    accuracy /= times_to_repeat
    return accuracy
lambdas = np.logspace(-5,3,100)

#Ridge Regression
ridge_accuracy = calc_accuracy(Ridge, lambdas)

#Lasso Regression 
lasso_accuracy = calc_accuracy(Lasso, lambdas)

In [19]:
#Elastic Regression
K = range(0,11,1)
accuracy = np.zeros((len(lambdas),len(K)))
for loop in range(times_to_repeat):
    err = 0
    for acc_index, acc in enumerate(K):
        for n_index, n in enumerate(lambdas):
            x_train, x_test,y_train, y_test = train_test_split(x_one_hot,y,test_size=0.25)
            model = ElasticNet(n, l1_ratio=acc).fit(x_train,y_train)
            y_pred =  model.predict(x_test)
            accuracy[n_index,acc_index] += (np.sum(y_test - y_pred)**2) / len(y_test)
accuracy /= times_to_repeat
elastic_accuracy = accuracy

In [22]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = range(1,15,3)
# Minimum number of samples required at each leaf node
min_samples_leaf = range(1,304,5)
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [28]:
x_train, x_test,y_train, y_test = train_test_split(x_one_hot,y,test_size=0.25, random_state=0)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(x_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.8min finished


{'n_estimators': 500,
 'min_samples_split': 13,
 'min_samples_leaf': 6,
 'max_features': 'auto',
 'max_depth': 50,
 'bootstrap': True}

In [29]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    accuracy = 100 - (100 * np.mean(errors / test_labels))
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [30]:

# Test if random model is actually better than the base model

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(x_train, y_train)
base_accuracy = evaluate(base_model, x_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Model Performance
Average Error: 2.8567 degrees.
Accuracy = 73.52%.
Model Performance
Average Error: 2.8234 degrees.
Accuracy = 91.35%.
Improvement of 24.25%.


In [33]:
#Random Forest
param_grid = {
    'min_samples_leaf': range(1,10),
    'max_depth': range(40,66),
    'min_samples_split': range(6,19),
    'n_estimators': range(400,600,20),
}

rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)

# Fit the grid search model
grid_search.fit(x_train, y_train)


Fitting 5 folds for each of 30420 candidates, totalling 152100 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 15.1min


In [ ]:
print(f'best estimator: {cv_grid.best_estimator_}')
print('Improvement of {:0.2f}% over the random estimator'.format( 100 * (grid_accuracy - random_accuracy) / random_accuracy))

In [ ]:



rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)# Train the model on training data
rf.fit(train_features, train_labels);

print(f'best estimator: {cv_grid.best_estimator_}')